<a href="https://colab.research.google.com/github/odddkidout/basic-term-GPU-model/blob/master/v0.2%20vs%20matthew%20pull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

edited code by me

In [28]:

import jax.numpy as jnp
import jax.numpy as np
from jax import jit,lax

"""Qx is mortality rate of the population"""
q = np.array([0,0.001,0.002,0.003,0.003,0.004,0.004,0.005,0.007,0.009,0.011])

"""Wx is Surrender rate of the population"""
w = np.array([0,0.05,0.07,0.08,0.10,0.14,0.20,0.20,0.20,0.10,0.04])

"""Yield curve"""
Yields = np.array([0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02])


""" We define some MPFs for policy information like premium, sum assured, Term , smoker status. this can be modified if an actuary wants to add more features"""
MPFs  = np.array(([10,25000,5,1],[100,25000,10,0],[80,20000,10,0],[80,20000,10,1]))
@jit
def calc(x,q,w,timestamp):
    
    numforce_v2 = jnp.cumprod(1-jnp.add(q,w))
    premium = jnp.multiply(x[0],numforce_v2)
    claim = jnp.multiply(x[1],jnp.multiply(numforce_v2,q))
    netCashflow = jnp.subtract(premium,claim)
    return (jnp.sum(jnp.multiply(netCashflow,jnp.float_power((1 + Yields)[x[2]], timestamp))))
    

for x in MPFs:
    qtemp = q[:x[2]]
    wtemp = w[:x[2]]
    timestamp = jnp.arange(x[2])
    %timeit -n 1 calc(x,qtemp,wtemp,-timestamp)


The slowest run took 10534.30 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 8.72 µs per loop
The slowest run took 10683.87 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 9.59 µs per loop
The slowest run took 7.02 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 8.66 µs per loop
The slowest run took 6.71 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 9.07 µs per loop


In [29]:
import jax.numpy as jnp

q = jnp.array([0.001,0.002,0.003,0.003,0.004,0.004,0.005,0.007,0.009,0.011])
w = jnp.array([0.05,0.07,0.08,0.10,0.14,0.20,0.20,0.20,0.10,0.04])
interest_rate = .02
face_value = 500_000
annual_premium = 3_000

# Policies in force starts at one, we add one to the beginning of the array
def calc():
    pols_if = jnp.concatenate([jnp.ones(1), jnp.cumprod((1-q)*(1-w))])
    timesteps = jnp.arange(pols_if.shape[0])
    # No cashflows out of the policy at time 0, add a zero to the beginning of the array
    claims = pols_if * jnp.concatenate([jnp.zeros(1), q]) * face_value
    premiums = pols_if * annual_premium

    net_cashflows = (premiums - claims)
    discount_factors = jnp.float_power(1 + interest_rate, -timesteps)
    discounted_cashflows = net_cashflows * discount_factors
    NPV = jnp.sum(discounted_cashflows)
    return NPV
  
%timeit -n 1 calc()

1 loop, best of 5: 1.84 ms per loop
